# DSPy on Azure OpenAI - 241026

In [ ]:
# Test AzureOpenAI endpoints 
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

deployment='gpt-4o-mini-eastus-071'

    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
    )
        
# Send a completion call to generate an answer
completion = client.chat.completions.create(
    model=deployment,
    messages = [
        {
        "role": "system",
        "content": "You are an MIT PhD in Physics, specializing in quantum physics."
        },
        {
        "role": "user",
        "content": "What is a black hole?"
        }
    ]
    # max_tokens=4096
)

#print(completion.model_dump_json(indent=2))
content = completion.choices[0].message.content
print(content)
print(len(content))

In [17]:
import os 
import dspy
from dotenv import dotenv_values, load_dotenv

load_dotenv()

config=dotenv_values(".env")
azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
embedding_model="text-embedding-3-small-eastus"
deployment='gpt-4o-mini-eastus-0718'



turbo = dspy.AzureOpenAI(
    api_key=api_key,
    api_version= "2024-06-01",
    api_base=azure_endpoint,
    model=deployment,
)

dspy.configure(lm=turbo)


In [ ]:
#Test DSpy with AzureOpenAI 01
qa = dspy.Predict('question: str -> response: str')
qa(question="What is a blackhole?").response

# Create the Contracts 

In [ ]:
import os
import pandas as pd
import random
import concurrent.futures
from openai import AzureOpenAI
from dotenv import load_dotenv
from time import sleep
from tqdm import tqdm

# Load environment variables
load_dotenv()

# Define the OpenAI client
deployment = 'gpt-4o-eastus-0806'

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
    )

# Create a contracts folder if it doesn't already exist
if not os.path.exists("contracts"):
    os.makedirs("contracts")

# Define clause types for the contract
clauses = [
    "Non-Disclosure Agreement (NDA) clause, which ensures confidentiality and protects sensitive business information.",
    "Termination Clause, detailing conditions under which either party may terminate the agreement.",
    "Indemnity Clause, covering liabilities, damages, and third-party claims arising from contract performance.",
    "Force Majeure Clause, specifying conditions where obligations are waived due to unforeseen events.",
    "Governing Law Clause, identifying the legal jurisdiction governing the contract.",
    "Arbitration Clause, defining dispute resolution through arbitration rather than court litigation.",
    "Confidentiality Agreement, ensuring restricted access to proprietary data and business secrets.",
    "Data Protection Clause, detailing how personal and sensitive data must be handled and secured."
]

# Function to make a single API call to generate content for multiple clauses
def generate_clauses(clause_types):
    messages = [{"role": "system", "content": "You are a legal assistant generating detailed contract clauses."}]
    for clause_type in clause_types:
        messages.append(
            {"role": "user", "content": f"Generate an 800-word section for a legal contract that includes a sophisticated {clause_type} with extensive legal language."}
        )
    try:
        completion = client.chat.completions.create(
            model=deployment,
            messages=messages,
            max_tokens=3000  # Approximate max tokens for multiple clauses
        )
        return completion.choices[0].message.content.strip().split("\n\n")
    except Exception as e:
        print(f"Error generating clauses for types '{clause_types}': {e}")
        return [""] * len(clause_types)

# Function to generate a complete contract and save it to a file
def generate_and_save_contract(contract_num):
    contract_sections = []
    labels = {}

    # Randomly decide if each clause will be "Present" or "Absent" and call generate_clauses in batches
    clause_requests = []
    for clause_type in clauses:
        if random.choice([True, False]):
            clause_requests.append(clause_type)
            labels[clause_type] = "Present"
        else:
            labels[clause_type] = "Absent"

    # Generate clauses in batches
    if clause_requests:
        clause_texts = generate_clauses(clause_requests)
        for text in clause_texts:
            contract_sections.append(text)

    # Generate additional filler sections to reach approx 8000 tokens
    filler_texts = generate_clauses(["General Contract Terms and Conditions"] * 5)
    contract_sections.extend(filler_texts)

    # Combine all sections into one contract text
    contract_text = " ".join(contract_sections)

    # Define the filename with proper numbering
    filename = f"contracts/contract{contract_num:02}.txt"

    # Write the contract to a file
    with open(filename, "w", encoding="utf-8") as file:
        file.write(contract_text)

    # Append the labels to be saved later in a DataFrame
    return {
        "filename": filename,
        "contains_nda": labels.get("Non-Disclosure Agreement (NDA) clause, which ensures confidentiality and protects sensitive business information.", "Absent"),
        "contains_termination": labels.get("Termination Clause, detailing conditions under which either party may terminate the agreement.", "Absent"),
        "contains_indemnity": labels.get("Indemnity Clause, covering liabilities, damages, and third-party claims arising from contract performance.", "Absent"),
        "contains_force_majeure": labels.get("Force Majeure Clause, specifying conditions where obligations are waived due to unforeseen events.", "Absent"),
        "contains_governing_law": labels.get("Governing Law Clause, identifying the legal jurisdiction governing the contract.", "Absent"),
        "contains_arbitration": labels.get("Arbitration Clause, defining dispute resolution through arbitration rather than court litigation.", "Absent"),
        "contains_confidentiality": labels.get("Confidentiality Agreement, ensuring restricted access to proprietary data and business secrets.", "Absent"),
        "contains_data_protection": labels.get("Data Protection Clause, detailing how personal and sensitive data must be handled and secured.", "Absent")
    }

# Function to run contract generation in parallel
def generate_contracts(num_contracts=50):
    contract_labels = []
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(generate_and_save_contract, i + 1) for i in range(num_contracts)]
        
        for future in tqdm(concurrent.futures.as_completed(futures), total=num_contracts, desc="Generating contracts"):
            result = future.result()
            contract_labels.append(result)

    return contract_labels

# Generate contracts and gather labels
labels = generate_contracts(50)

# Save the labels to a DataFrame and store it in the contracts folder
df = pd.DataFrame(labels)
df.to_csv("contracts/contract_labels.csv", index=False)
print("Contract labels saved as contracts/contract_labels.csv")


# Eval with gpt-4o-mini

In [1]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
import pandas as pd
import random
from tqdm import tqdm


# Load environment variables
load_dotenv()

# Initialize Azure OpenAI client
deployment = 'gpt-4o-mini-eastus-0718'
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_EASTUS_API_KEY"),  
    api_version="2024-06-01",
    azure_endpoint=os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
)
# Define all clauses to check

clauses_to_check = [
    "Non-Disclosure Agreement (NDA)",
    "Termination Clause",
    "Indemnity Clause",
    "Force Majeure Clause",
    "Governing Law Clause",
    "Arbitration Clause",
    "Confidentiality Agreement",
    "Data Protection Clause"
]

# Load actual labels
df = pd.read_csv("contracts/contract_labels.csv")

# Define a function to query the model for each clause
def query_model_for_clauses(contract_text):
    responses = {}
    
    for clause in clauses_to_check:
        print(f"\n[INFO] Querying model to check presence of '{clause}' in the contract.")
        
        # Ask the model to classify whether the specific clause is present or absent
        completion = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "system", "content": "You are a contract classification assistant."},
                {"role": "user", "content": f"Does the following contract contain a '{clause}'? Answer 'Present' or 'Absent'.\n\nContract: {contract_text}"}
            ]
        )
        
        content = completion.choices[0].message.content.strip()
        print(f"[DEBUG] Model response for '{clause}': '{content}'")
        
        # Process response to standardize it to 'Present' or 'Absent'
        if "present" in content.lower():
            responses[clause] = "Present"
        elif "absent" in content.lower():
            responses[clause] = "Absent"
        else:
            responses[clause] = "Unknown"  # Handle unexpected responses
        
    return responses

# Run the multi-label classification task on each contract
predictions = []
for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing contracts"):
    # Read contract text from each file
    with open(row["filename"], "r", encoding="utf-8") as file:
        contract_text = file.read()
    
    # Actual labels for the contract
    actual_labels = {
        "Non-Disclosure Agreement (NDA)": row["contains_nda"],
        "Termination Clause": row["contains_termination"],
        "Indemnity Clause": row["contains_indemnity"],
        "Force Majeure Clause": row["contains_force_majeure"],
        "Governing Law Clause": row["contains_governing_law"],
        "Arbitration Clause": row["contains_arbitration"],
        "Confidentiality Agreement": row["contains_confidentiality"],
        "Data Protection Clause": row["contains_data_protection"]
    }
    
    # Predicted labels
    predicted_labels = query_model_for_clauses(contract_text)
    
    # Check accuracy for each clause
    is_correct = {clause: actual_labels[clause] == predicted_labels[clause] for clause in actual_labels}
    print(f"\n[INFO] Actual Labels: {actual_labels}")
    print(f"[INFO] Predicted Labels: {predicted_labels}")
    print(f"[RESULT] Clause-by-clause correctness: {is_correct}\n{'-' * 50}")
    
    predictions.append({
        "filename": row["filename"],
        "actual_labels": actual_labels,
        "predicted_labels": predicted_labels,
        "is_correct": is_correct
    })

# Calculate overall accuracy for each clause
correct_counts = {clause: 0 for clause in clauses_to_check}
total_counts = {clause: 0 for clause in clauses_to_check}

for result in predictions:
    for clause, correct in result["is_correct"].items():
        total_counts[clause] += 1
        if correct:
            correct_counts[clause] += 1

accuracy = {clause: correct_counts[clause] / total_counts[clause] * 100 for clause in correct_counts}

# Display results
print("\n[FINAL RESULTS]")
for clause, acc in accuracy.items():
    print(f"Accuracy for '{clause}': {acc:.2f}%")

Processing contracts:   0%|          | 0/50 [00:00<?, ?it/s]


[INFO] Querying model to check presence of 'Non-Disclosure Agreement (NDA)' in the contract.
[DEBUG] Model response for 'Non-Disclosure Agreement (NDA)': 'Absent'

[INFO] Querying model to check presence of 'Termination Clause' in the contract.
[DEBUG] Model response for 'Termination Clause': 'Present'

[INFO] Querying model to check presence of 'Indemnity Clause' in the contract.
[DEBUG] Model response for 'Indemnity Clause': 'Present'

[INFO] Querying model to check presence of 'Force Majeure Clause' in the contract.
[DEBUG] Model response for 'Force Majeure Clause': 'Present'

[INFO] Querying model to check presence of 'Governing Law Clause' in the contract.
[DEBUG] Model response for 'Governing Law Clause': 'Present'

[INFO] Querying model to check presence of 'Arbitration Clause' in the contract.
[DEBUG] Model response for 'Arbitration Clause': 'Absent'

[INFO] Querying model to check presence of 'Confidentiality Agreement' in the contract.
[DEBUG] Model response for 'Confidentia

Processing contracts:   2%|▏         | 1/50 [00:03<03:11,  3.90s/it]

[DEBUG] Model response for 'Data Protection Clause': 'Present'

[INFO] Actual Labels: {'Non-Disclosure Agreement (NDA)': 'Present', 'Termination Clause': 'Present', 'Indemnity Clause': 'Present', 'Force Majeure Clause': 'Absent', 'Governing Law Clause': 'Absent', 'Arbitration Clause': 'Present', 'Confidentiality Agreement': 'Absent', 'Data Protection Clause': 'Present'}
[INFO] Predicted Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Present', 'Indemnity Clause': 'Present', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Absent', 'Confidentiality Agreement': 'Present', 'Data Protection Clause': 'Present'}
[RESULT] Clause-by-clause correctness: {'Non-Disclosure Agreement (NDA)': False, 'Termination Clause': True, 'Indemnity Clause': True, 'Force Majeure Clause': False, 'Governing Law Clause': False, 'Arbitration Clause': False, 'Confidentiality Agreement': False, 'Data Protection Clause': True}
--------------------

Processing contracts:   4%|▍         | 2/50 [00:06<02:42,  3.38s/it]

...

[DEBUG] Model response for 'Data Protection Clause': 'Absent'

[INFO] Actual Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Present', 'Indemnity Clause': 'Present', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Absent', 'Confidentiality Agreement': 'Absent', 'Data Protection Clause': 'Absent'}
[INFO] Predicted Labels: {'Non-Disclosure Agreement (NDA)': 'Present', 'Termination Clause': 'Present', 'Indemnity Clause': 'Absent', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Present', 'Confidentiality Agreement': 'Present', 'Data Protection Clause': 'Absent'}
[RESULT] Clause-by-clause correctness: {'Non-Disclosure Agreement (NDA)': False, 'Termination Clause': True, 'Indemnity Clause': False, 'Force Majeure Clause': True, 'Governing Law Clause': True, 'Arbitration Clause': False, 'Confidentiality Agreement': False, 'Data Protection Clause': True}
-----------------------

Processing contracts:  98%|█████████▊| 49/50 [03:14<00:03,  3.86s/it]

[DEBUG] Model response for 'Data Protection Clause': '**Present**'

[INFO] Actual Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Present', 'Indemnity Clause': 'Present', 'Force Majeure Clause': 'Absent', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Present', 'Confidentiality Agreement': 'Present', 'Data Protection Clause': 'Present'}
[INFO] Predicted Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Present', 'Indemnity Clause': 'Present', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Absent', 'Confidentiality Agreement': 'Present', 'Data Protection Clause': 'Present'}
[RESULT] Clause-by-clause correctness: {'Non-Disclosure Agreement (NDA)': True, 'Termination Clause': True, 'Indemnity Clause': True, 'Force Majeure Clause': False, 'Governing Law Clause': True, 'Arbitration Clause': False, 'Confidentiality Agreement': True, 'Data Protection Clause': True}
------------------

Processing contracts: 100%|██████████| 50/50 [03:19<00:00,  3.99s/it]

[DEBUG] Model response for 'Data Protection Clause': 'Absent'

[INFO] Actual Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Absent', 'Indemnity Clause': 'Absent', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Absent', 'Arbitration Clause': 'Absent', 'Confidentiality Agreement': 'Absent', 'Data Protection Clause': 'Absent'}
[INFO] Predicted Labels: {'Non-Disclosure Agreement (NDA)': 'Absent', 'Termination Clause': 'Present', 'Indemnity Clause': 'Absent', 'Force Majeure Clause': 'Present', 'Governing Law Clause': 'Present', 'Arbitration Clause': 'Present', 'Confidentiality Agreement': 'Present', 'Data Protection Clause': 'Absent'}
[RESULT] Clause-by-clause correctness: {'Non-Disclosure Agreement (NDA)': True, 'Termination Clause': False, 'Indemnity Clause': True, 'Force Majeure Clause': True, 'Governing Law Clause': False, 'Arbitration Clause': False, 'Confidentiality Agreement': False, 'Data Protection Clause': True}
---------------------------

In [ ]:
accuracy.items():
    print(f"Accuracy for '{clause}': {acc:.2f}%")

# Optimise with DSPy

In [1]:
import os
import pandas as pd
import dspy
import random
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from tqdm import tqdm
from dotenv import load_dotenv
import logging
import openai
from openai import AzureOpenAI

# Import assertion modules
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
from dspy.predict import Retry

# Set up logging to display DSPy logs
dspy.logger.setLevel(logging.INFO)

# Load environment variables
load_dotenv()

azure_endpoint = os.getenv("AZURE_OPENAI_API_EASTUS_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_EASTUS_API_KEY")
deployment = 'gpt-4o-mini'

# Initialize Azure OpenAI client
client = dspy.AzureOpenAI(
    api_key=api_key,
    api_version="2024-06-01",
    api_base=azure_endpoint,
    model=deployment
)

clientx = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version="2024-06-01",
)
# Set the client as the default language model in DSPy settings
dspy.settings.configure(lm=client, temperature=0)

# Define the clauses to check
clauses_to_check = [
    "Non-Disclosure Agreement (NDA)",
    "Termination Clause",
    "Indemnity Clause",
    "Force Majeure Clause",
    "Governing Law Clause",
    "Arbitration Clause",
    "Confidentiality Agreement",
    "Data Protection Clause"
]

# Updated ClauseClassifierSignature with explicit instructions
class ClauseClassifierSignature(dspy.Signature):
    """Determine if the specified clause is present in the contract.

    Instructions:
    - Read the contract text carefully.
    - Check for the presence of the clause specified in 'clause_check'.
    - Respond with 'present' if the clause is present.
    - Respond with 'absent' if the clause is not present.
    - Provide your answer as either 'present' or 'absent' only, without any additional text.
    """

    contract_text = dspy.InputField(desc="The full text of the contract document.")
    clause_check = dspy.InputField(desc="The type of clause to check for.")
    answer = dspy.OutputField(desc="Respond with 'present' or 'absent'.")

# Remove manually created few-shot examples
# few_shot_examples = [
#     {
#         'contract_text': "The Employee agrees not to disclose any confidential information obtained during the term of employment.",
#         'clause_check': "Confidentiality",
#         'answer': "present"
#     },
#     {
#         'contract_text': "This agreement does not include any terms related to competition or non-competition.",
#         'clause_check': "Non-Compete",
#         'answer': "absent"
#     },
#     # Add more examples as needed
# ]

# Define the classifier program with the capability to include few-shot examples
class ContractClauseClassifier(dspy.Module):
    def __init__(self, client, clientx, deployment):
        super().__init__()
        self.client = client       # dspy client
        self.clientx = clientx     # direct Azure OpenAI client
        self.deployment = deployment
        self.demos = []            # Initialize empty demos; optimizer will set this

    def forward(self, contract_text, clause_check):
        # Prepare the few-shot examples
        few_shot_messages = []
        for example in self.demos:
            few_shot_messages.append({
                "role": "user",
                "content": f"""Determine if the following clause is present in this contract.

Clause: {example['clause_check']}

Contract Text: {example['contract_text']}

Provide your answer as 'present' or 'absent' only."""
            })
            few_shot_messages.append({
                "role": "assistant",
                "content": example['answer']
            })

        # Now build the messages including the few-shot examples
        messages = [
            {"role": "system", "content": "You are a legal expert analyzing the presence of various clauses in a contract. Answer 'present' or 'absent' only."}
        ] + few_shot_messages + [
            {
                "role": "user",
                "content": f"""Determine if the following clause is present in this contract.

Clause: {clause_check}

Contract Text: {contract_text}

Provide your answer as 'present' or 'absent' only."""
            }
        ]

        try:
            completion = self.clientx.chat.completions.create(
                model=self.deployment,
                messages=messages,
                max_tokens=5  # Limit tokens to encourage short answers
            )
            response = completion.choices[0].message.content.strip().lower()
            if response in ["present", "absent"]:
                return {"answer": response}
            else:
                return {"answer": "unknown"}
        except Exception as e:
            print(f"Error processing contract: {e}")
            return {"answer": "unknown"}

classifier = ContractClauseClassifier(client, clientx, deployment)

# Define the metric for evaluation
def clause_classification_metric(example, pred, trace=None):
    actual = example['answer'].strip().lower()
    predicted = pred['answer'].strip().lower() if pred and 'answer' in pred else ""

    valid_labels = ["present", "absent"]
    if actual not in valid_labels:
        return 0  # Cannot evaluate if actual label is unknown

    if predicted not in valid_labels:
        return 0  # Prediction is invalid

    return int(actual == predicted)

# Initialize the optimizer with the metric
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    metric=clause_classification_metric,
    max_bootstrapped_demos=8,
    max_labeled_demos=8,
    num_candidate_programs=10,
    num_threads=4,
    teacher_settings=dict(lm=client)
)

# Load your data
df = pd.read_csv('contracts/contract_labels.csv')  # Update the path as needed

# Prepare the examples
examples = []

clause_to_label_column = {
    "Non-Disclosure Agreement (NDA)": "contains_nda",
    "Termination Clause": "contains_termination",
    "Indemnity Clause": "contains_indemnity",
    "Force Majeure Clause": "contains_force_majeure",
    "Governing Law Clause": "contains_governing_law",
    "Arbitration Clause": "contains_arbitration",
    "Confidentiality Agreement": "contains_confidentiality",
    "Data Protection Clause": "contains_data_protection"
}

for idx, row in df.iterrows():
    # Read contract text
    try:
        with open(row["filename"], "r", encoding="utf-8") as file:
            contract_text = file.read()
    except FileNotFoundError:
        print(f"File '{row['filename']}' not found.")
        continue

    for clause in clauses_to_check:
        label_column = clause_to_label_column.get(clause)
        if label_column not in df.columns:
            actual_label = "unknown"
        else:
            actual_label = str(row[label_column]).strip().lower()
            if actual_label not in ["present", "absent"]:
                actual_label = "unknown"

        # Create example with specified inputs
        example = dspy.Example(
            contract_text=contract_text,
            clause_check=clause,
            answer=actual_label
        ).with_inputs('contract_text', 'clause_check')
        examples.append(example)

# Shuffle and split examples
random.shuffle(examples)
split_ratio = 0.8
split_index = int(len(examples) * split_ratio)

trainset = examples[:split_index]
valset = examples[split_index:]

print(f"Total examples: {len(examples)}")
print(f"Training examples: {len(trainset)}")
print(f"Validation examples: {len(valset)}")

if not valset:
    raise ValueError("Validation set is empty. Cannot proceed.")

# Toggle to run bootstrapping from scratch
RUN_FROM_SCRATCH = True

# Train or load the model
if RUN_FROM_SCRATCH:
    clause_classifier = bootstrap_optimizer.compile(
        student=classifier,
        trainset=trainset,
        valset=valset
    )
    clause_classifier.save("optimized_fewshot_classifier.json")
else:
    clause_classifier = ContractClauseClassifier(client, clientx, deployment)
    clause_classifier.load("optimized_fewshot_classifier.json")

# Predict and evaluate

import json

class ExampleEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, dspy.Example):
            return obj._store  # or vars(obj) or obj.__dict__
        return json.JSONEncoder.default(self, obj)
    
def example_to_dict(example):
    return {
        'inputs': example.inputs,
        'outputs': example.outputs,
        # Include any other relevant fields
    }

from dspy.evaluate.evaluate import Evaluate as BaseEvaluate

class Evaluate(BaseEvaluate):
    def __call__(self, *args, **kwargs):
        # Override methods to prevent serialization of Examples
        # Implement custom logic here
        pass

predictions = []

for idx, row in tqdm(df.iterrows(), total=df.shape[0], desc="Processing contracts"):
    # Read contract text
    try:
        with open(row["filename"], "r", encoding="utf-8") as file:
            contract_text = file.read()
    except FileNotFoundError:
        print(f"File '{row['filename']}' not found.")
        continue

    actual_labels = {}
    predicted_labels = {}

    for clause in clauses_to_check:
        label_column = clause_to_label_column.get(clause)
        if label_column not in df.columns:
            actual_label = "unknown"
            continue
        else:
            actual_label = str(row[label_column]).strip().lower()
            if actual_label not in ["present", "absent"]:
                actual_label = "unknown"
                continue
        actual_labels[clause] = actual_label

        # Make prediction
        prediction = clause_classifier(contract_text=contract_text, clause_check=clause)
        predicted_label = prediction['answer'].strip().lower() if prediction and 'answer' in prediction else "unknown"
        predicted_labels[clause] = predicted_label

    # Track correctness per clause
    is_correct = {
        clause: actual_labels[clause] == predicted_labels[clause]
        for clause in clauses_to_check if actual_labels.get(clause, "unknown") != "unknown"
    }
    predictions.append({
        "filename": row["filename"],
        "actual_labels": actual_labels,
        "predicted_labels": predicted_labels,
        "is_correct": is_correct
    })

# Calculate and display results
correct_counts = {clause: 0 for clause in clauses_to_check}
total_counts = {clause: 0 for clause in clauses_to_check}

for result in predictions:
    for clause in clauses_to_check:
        actual_label = result["actual_labels"].get(clause, "unknown")
        if actual_label != "unknown":
            total_counts[clause] += 1
            if result["is_correct"].get(clause, False):
                correct_counts[clause] += 1

accuracy = {
    clause: (correct_counts[clause] / total_counts[clause] * 100) if total_counts[clause] > 0 else 0
    for clause in clauses_to_check
}

print("\n[FINAL RESULTS]")
for clause, acc in accuracy.items():
    print(f"Accuracy for '{clause}': {acc:.2f}%")


/Users/ozgurguler/Developer/Projects/dspy/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
Total examples: 400
Training examples: 320
Validation examples: 80


Average Metric: 48 / 80  (60.0): 100%|██████████| 80/80 [00:09<00:00,  8.49it/s]


New best score: 60.0 for seed -3
Scores so far: [60.0]
Best score so far: 60.0


Average Metric: 48 / 80  (60.0): 100%|██████████| 80/80 [00:09<00:00,  8.60it/s]


...

Bootstrapped 8 full traces after 11 examples in round 0.


Average Metric: 48 / 80  (60.0): 100%|██████████| 80/80 [00:09<00:00,  8.37it/s]


Scores so far: [60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 62.5, 60.0]
Best score so far: 62.5
13 candidate programs found.


Processing contracts: 100%|██████████| 50/50 [03:09<00:00,  3.78s/it]


[FINAL RESULTS]
Accuracy for 'Non-Disclosure Agreement (NDA)': 60.00%
Accuracy for 'Termination Clause': 50.00%
Accuracy for 'Indemnity Clause': 46.00%
Accuracy for 'Force Majeure Clause': 50.00%
Accuracy for 'Governing Law Clause': 52.00%
Accuracy for 'Arbitration Clause': 52.00%
Accuracy for 'Confidentiality Agreement': 58.00%
Accuracy for 'Data Protection Clause': 100.00%
